In [1]:
import sys
sys.path.append('../src')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data.load_data import load_data_from_csv
from trade.trading_strategy import moving_average_strategy
from trade.backtest import simulate_trades_with_allocation
from utils import *


tickers = ['GOOGL', 'AMZN', 'AAPL', 'PEP', 'JNJ', 'PFE', 'MRK', 'ABBV', 'PG', 'KO',
           'WMT', 'JPM', 'BAC', 'GS', 'V', 'XOM', 'CVX', 'COP', 'BP', 'BA',
           'MMM', 'HON', 'GE', 'T', 'VZ', 'TMUS', 'HSY', 'DUK', 'SO', 'EXC', 'AEP',
           'AMT', 'PLD', 'SPG', 'BHP', 'RIO', 'VALE', 'FCX']
start_date = '2010-01-01'
end_date = '2023-10-23'
sequence_length = 50
window_sizes = [50, 40, 30]

Processing GOOGL...


2024-08-05 01:07:32.057832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 01:07:32.059232: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


22/22 [==============================] - 2s 43ms/step
Processing AMZN...
22/22 [==============================] - 2s 44ms/step
Processing AAPL...
22/22 [==============================] - 1s 38ms/step
Processing PEP...
22/22 [==============================] - 1s 40ms/step
Processing JNJ...
22/22 [==============================] - 1s 42ms/step
Processing PFE...
22/22 [==============================] - 1s 41ms/step
Processing MRK...
22/22 [==============================] - 1s 38ms/step
Processing ABBV...
17/17 [==============================] - 1s 37ms/step
Processing PG...
22/22 [==============================] - 1s 41ms/step
Processing KO...
22/22 [==============================] - 1s 46ms/step
Processing WMT...
22/22 [==============================] - 1s 42ms/step
Processing JPM...
22/22 [==============================] - 2s 46ms/step
Processing BAC...
22/22 [==============================] - 1s 39ms/step
Processing GS...
22/22 [==============================] - 1s 40ms/step
Processing

In [2]:
predictions, actuals = get_predictions(tickers, start_date, end_date, sequence_length, folder_path='models')

trading_signals = moving_average_strategy(predictions)

portfolio_returns = simulate_trades_with_allocation(predictions, actuals, trading_signals, initial_capital=100000, allocation_percentage=0.1, slippage_rate=0.02, transaction_cost=0.005)



Processing GOOGL...
22/22 [==============================] - 2s 69ms/step
Processing AMZN...
22/22 [==============================] - 2s 52ms/step
Processing AAPL...
22/22 [==============================] - 2s 84ms/step
Processing PEP...
22/22 [==============================] - 3s 81ms/step
Processing JNJ...
22/22 [==============================] - 2s 88ms/step
Processing PFE...
22/22 [==============================] - 3s 90ms/step
Processing MRK...
22/22 [==============================] - 3s 97ms/step
Processing ABBV...
17/17 [==============================] - 2s 84ms/step
Processing PG...
22/22 [==============================] - 3s 89ms/step
Processing KO...
22/22 [==============================] - 2s 85ms/step
Processing WMT...
22/22 [==============================] - 2s 79ms/step
Processing JPM...
22/22 [==============================] - 2s 79ms/step
Processing BAC...
22/22 [==============================] - 2s 50ms/step
Processing GS...
22/22 [==============================] - 2s 

In [3]:
targeted_dates = [
    "2021-11-15", "2021-12-02", "2022-01-24", "2022-02-08",
    "2023-04-17", "2023-05-10", "2023-06-13", "2023-07-11"
]

scenarios = {
    "scenario_1": 0.0,   # Same as previous price
    "scenario_2": 0.005, # 0.5% higher
    "scenario_3": -0.005 # 0.5% lower
}

def get_targeted_indices_for_ticker(ticker, start_date, end_date, sequence_length, targeted_dates):
    _, _, X_test, _, _, _, x_min, _ = preprocess_data(ticker, start_date, end_date, sequence_length)
    
    full_dates_range = pd.date_range(start=start_date, end=end_date)
    test_dates_range = full_dates_range[-X_test.shape[0]:]  
    
    targeted_indices = []
    for date in targeted_dates:
        date = pd.Timestamp(date)  
        idx = np.where(test_dates_range == date)[0]
        if idx.size != 0:  
            targeted_indices.append(idx[0])
    
    return targeted_indices

def perform_scenario_attack(ticker, start_date, end_date, sequence_length, attack_indices, drop_percentage, folder_path="models"):
    model_path = os.path.join(folder_path, f"{ticker}_model.h5")
    model = load_model(model_path)

    _, _, X_test, y_test, y_min, y_max, x_min, x_max = preprocess_data(ticker, start_date, end_date, sequence_length)

    attacked_X_test = X_test.copy()
    y_actual = y_test * (y_max - y_min) + y_min

    for attack_day in attack_indices:
        previous_price = y_actual[attack_day-1]
        if drop_percentage == 0.0:
            new_price = previous_price
        else:
            new_price = previous_price * (1 + drop_percentage)
        attacked_X_test[attack_day, -1, 3] = (new_price - y_min) / (y_max - y_min)

    y_pred = model.predict(attacked_X_test)
    y_pred_rescaled = y_pred * (y_max - y_min) + y_min

    predictions = {ticker: y_pred_rescaled.flatten()}
    actuals = {ticker: y_actual.flatten()}

    return predictions, actuals

def calculate_cumulative_returns(predictions, actuals):
    returns = (predictions / actuals) - 1
    cumulative_returns = np.cumsum(returns)
    return cumulative_returns

def analyze_scenarios_for_targeted_days(tickers, start_date, end_date, sequence_length, targeted_indices):
    baseline_predictions, baseline_actuals = get_predictions(tickers, start_date, end_date, sequence_length)
    
    baseline_cumulative_returns = calculate_cumulative_returns(baseline_predictions['GOOGL'], baseline_actuals['GOOGL'])
    
    results = { "scenario_1": [], "scenario_2": [], "scenario_3": [] }

    for attack_day in targeted_indices:
        for scenario_name, drop_percentage in scenarios.items():
            predictions_after_attack, actuals_after_attack = perform_scenario_attack('GOOGL', start_date, end_date, sequence_length, [attack_day], drop_percentage)
            
            combined_predictions = baseline_predictions.copy()
            combined_predictions['GOOGL'] = predictions_after_attack['GOOGL']
            
            cumulative_returns_after_attack = calculate_cumulative_returns(combined_predictions['GOOGL'], baseline_actuals['GOOGL'])
            
            delta_cr = cumulative_returns_after_attack - baseline_cumulative_returns
            delta_percentage = (delta_cr / 0.20) * 100
            
            results[scenario_name].append((attack_day, delta_cr[-1], delta_percentage[-1]))

    if not os.path.exists('results'):
        os.makedirs('results')

    with open('results/concea_scenarios.txt', 'w') as f:
        for scenario, data in results.items():
            f.write(f"\nResults for {scenario}:\n")
            for attack_day, delta_cr, delta_percentage in data:
                f.write(f"Day {attack_day}: CR change: {delta_cr}, Delta %: {delta_percentage}\n")

    for scenario, data in results.items():
        print(f"\nResults for {scenario}:")
        for attack_day, delta_cr, delta_percentage in data:
            print(f"Day {attack_day}: CR change: {delta_cr}, Delta %: {delta_percentage}")

targeted_indices = get_targeted_indices_for_ticker('GOOGL', start_date, end_date, sequence_length, targeted_dates)

analyze_scenarios_for_targeted_days(tickers, start_date, end_date, sequence_length, targeted_indices)

Processing GOOGL...
22/22 [==============================] - 2s 63ms/step
Processing AMZN...
22/22 [==============================] - 2s 61ms/step
Processing AAPL...
22/22 [==============================] - 2s 48ms/step
Processing PEP...
22/22 [==============================] - 2s 48ms/step
Processing JNJ...
22/22 [==============================] - 2s 71ms/step
Processing PFE...
22/22 [==============================] - 2s 76ms/step
Processing MRK...
22/22 [==============================] - 2s 70ms/step
Processing ABBV...
17/17 [==============================] - 2s 77ms/step
Processing PG...
22/22 [==============================] - 2s 77ms/step
Processing KO...
22/22 [==============================] - 2s 62ms/step
Processing WMT...
22/22 [==============================] - 2s 62ms/step
Processing JPM...
22/22 [==============================] - 2s 71ms/step
Processing BAC...
22/22 [==============================] - 2s 37ms/step
Processing GS...
22/22 [==============================] - 2s 

In [5]:

# Define the targeted dates and scenarios
targeted_dates = [
    "2021-11-15", "2021-12-02", "2022-01-24", "2022-02-08",
    "2023-04-17", "2023-05-10", "2023-06-13", "2023-07-11"
]

scenarios = {
    "scenario_1": -0.10,  # 10% drop
    "scenario_2": -0.025, # 2.5% drop
    "scenario_3": -0.01   # 1% drop
}

def get_targeted_indices_for_ticker(ticker, start_date, end_date, sequence_length, targeted_dates):
    _, _, X_test, _, _, _, x_min, _ = preprocess_data(ticker, start_date, end_date, sequence_length)
    
    full_dates_range = pd.date_range(start=start_date, end=end_date)
    test_dates_range = full_dates_range[-X_test.shape[0]:]  
    
    targeted_indices = []
    for date in targeted_dates:
        date = pd.Timestamp(date)  
        idx = np.where(test_dates_range == date)[0]
        if idx.size != 0:  
            targeted_indices.append(idx[0])
    
    return targeted_indices

def perform_scenario_attack(ticker, start_date, end_date, sequence_length, attack_indices, drop_percentage, folder_path="models"):
    model_path = os.path.join(folder_path, f"{ticker}_model.h5")
    model = load_model(model_path)

    _, _, X_test, y_test, y_min, y_max, x_min, x_max = preprocess_data(ticker, start_date, end_date, sequence_length)

    attacked_X_test = X_test.copy()
    y_actual = y_test * (y_max - y_min) + y_min

    for attack_day in attack_indices:
        previous_price = y_actual[attack_day-1]
        if drop_percentage == 0.0:
            new_price = previous_price
        else:
            new_price = previous_price * (1 + drop_percentage)
        attacked_X_test[attack_day, -1, 3] = (new_price - y_min) / (y_max - y_min)

    y_pred = model.predict(attacked_X_test)
    y_pred_rescaled = y_pred * (y_max - y_min) + y_min

    predictions = {ticker: y_pred_rescaled.flatten()}
    actuals = {ticker: y_actual.flatten()}

    return predictions, actuals

def calculate_cumulative_returns(predictions, actuals):
    returns = (predictions / actuals) - 1
    cumulative_returns = np.cumsum(returns)
    return cumulative_returns

def analyze_scenarios_for_targeted_days(tickers, start_date, end_date, sequence_length, targeted_indices):
    baseline_predictions, baseline_actuals = get_predictions(tickers, start_date, end_date, sequence_length)
    
    baseline_cumulative_returns = calculate_cumulative_returns(baseline_predictions['GOOGL'], baseline_actuals['GOOGL'])
    
    results = { "scenario_1": [], "scenario_2": [], "scenario_3": [] }

    for attack_day in targeted_indices:
        for scenario_name, drop_percentage in scenarios.items():
            predictions_after_attack, actuals_after_attack = perform_scenario_attack('GOOGL', start_date, end_date, sequence_length, [attack_day], drop_percentage)
            
            combined_predictions = baseline_predictions.copy()
            combined_predictions['GOOGL'] = predictions_after_attack['GOOGL']
            
            cumulative_returns_after_attack = calculate_cumulative_returns(combined_predictions['GOOGL'], baseline_actuals['GOOGL'])
            
            delta_cr = cumulative_returns_after_attack - baseline_cumulative_returns
            delta_percentage = (delta_cr / 0.20) * 100
            
            results[scenario_name].append((attack_day, delta_cr[-1], delta_percentage[-1]))

    if not os.path.exists('results'):
        os.makedirs('results')

    with open('results/overestimate_scenarios.txt', 'w') as f:
        for scenario, data in results.items():
            f.write(f"\nResults for {scenario}:\n")
            for attack_day, delta_cr, delta_percentage in data:
                f.write(f"Day {attack_day}: CR change: {delta_cr}, Delta %: {delta_percentage}\n")

    for scenario, data in results.items():
        print(f"\nResults for {scenario}:")
        for attack_day, delta_cr, delta_percentage in data:
            print(f"Day {attack_day}: CR change: {delta_cr}, Delta %: {delta_percentage}")

targeted_indices = get_targeted_indices_for_ticker('GOOGL', start_date, end_date, sequence_length, targeted_dates)

analyze_scenarios_for_targeted_days(tickers, start_date, end_date, sequence_length, targeted_indices)

Processing GOOGL...
22/22 [==============================] - 2s 63ms/step
Processing AMZN...
22/22 [==============================] - 2s 54ms/step
Processing AAPL...
22/22 [==============================] - 2s 61ms/step
Processing PEP...
22/22 [==============================] - 2s 54ms/step
Processing JNJ...
22/22 [==============================] - 2s 63ms/step
Processing PFE...
22/22 [==============================] - 2s 48ms/step
Processing MRK...
22/22 [==============================] - 2s 43ms/step
Processing ABBV...
17/17 [==============================] - 1s 49ms/step
Processing PG...
22/22 [==============================] - 2s 53ms/step
Processing KO...
22/22 [==============================] - 2s 42ms/step
Processing WMT...
22/22 [==============================] - 2s 46ms/step
Processing JPM...
22/22 [==============================] - 2s 54ms/step
Processing BAC...
22/22 [==============================] - 2s 42ms/step
Processing GS...
22/22 [==============================] - 2s 